In [2]:
import pandas as pd

In [3]:
# Load cleaned data
candidates = pd.read_csv('../../data/candidates.csv')
employees = pd.read_csv('../outputs/summary_tables/employees_cleaned.csv')
employments = pd.read_csv('../outputs/summary_tables/employments_cleaned.csv')
positions = pd.read_csv('../../data/positions.csv')
recruitments = pd.read_csv('../../data/recruitments.csv')

In [5]:
# Standardize column names
for df in [candidates, employees, employments, positions, recruitments]:
    df.columns = df.columns.str.lower()

In [6]:
print("Recruitments columns:", recruitments.columns.tolist())
print("Candidates columns:", candidates.columns.tolist())
print("Employees columns:", employees.columns.tolist())
print("Employments columns:", employments.columns.tolist())
print("Positions columns:", positions.columns.tolist())


Recruitments columns: ['id', 'candidate_id', 'position_id', 'stage', 'stage_start_date', 'recruitment_channel', 'recruitment_cost', 'notes']
Candidates columns: ['id', 'first_name', 'last_name', 'email', 'phone']
Employees columns: ['id', 'first_name', 'last_name', 'gender', 'birth_date', 'hire_date', 'termination_date', 'status', 'position_id', 'manager_id']
Employments columns: ['id', 'employee_id', 'position_id', 'start_date', 'salary', 'bonus', 'pension_contribution', 'vacation_allowance']
Positions columns: ['id', 'title', 'job_level', 'department']


In [7]:
# Merge recruitment with candidates
recruit_df = pd.merge(
    recruitments, 
    candidates, 
    how='left', 
    left_on='candidate_id', 
    right_on='id', 
    suffixes=('', '_candidate')
)

print(recruit_df.columns.tolist())

['id', 'candidate_id', 'position_id', 'stage', 'stage_start_date', 'recruitment_channel', 'recruitment_cost', 'notes', 'id_candidate', 'first_name', 'last_name', 'email', 'phone']


In [ ]:
# Merge with positions to add job details
recruit_df = pd.merge(
    recruit_df,
    positions,
    how='left',
    left_on='position_id',
    right_on='id',
    suffixes=('', '_position')
)

# Drop redundant position ID from positions table
recruit_df.drop(columns=['id_position'], inplace=True)


print(recruit_df.columns.tolist())

['id', 'candidate_id', 'position_id', 'stage', 'stage_start_date', 'recruitment_channel', 'recruitment_cost', 'notes', 'id_candidate', 'first_name', 'last_name', 'email', 'phone', 'title', 'job_level', 'department']


In [ ]:
# Convert to string to avoid merge issues
recruit_df['candidate_id'] = recruit_df['candidate_id'].astype(str)
employees['id'] = employees['id'].astype(str)

# Add 'hired' flag
recruit_df['hired'] = recruit_df['candidate_id'].isin(employees['id'])

Average Recruitment Cost by Department

In [ ]:
avg_cost = (
    recruit_df.groupby('department')['recruitment_cost']
    .mean()
    .reset_index()
    .sort_values(by='recruitment_cost', ascending=False)
)

display(avg_cost.head())

,department,recruitment_cost
4,Management,2342.642857
0,Business Intelligence,2231.163265
1,Cybersecurity,2229.481818
3,Human Resources,2205.280899
5,Marketing,2196.483582


In [ ]:
# Make sure output directory exists first
import os
os.makedirs('../../outputs/summary_tables', exist_ok=True)

# Save summary tables
avg_cost.to_csv('../outputs/summary_tables/recruitment_cost_by_department.csv', index=False)

In [ ]:
# Convert date columns
employees['hire_date'] = pd.to_datetime(employees['hire_date'], errors='coerce')
employees['termination_date'] = pd.to_datetime(employees['termination_date'], errors='coerce')

Employee Tenure

In [18]:
# Convert date columns
employees['hire_date'] = pd.to_datetime(employees['hire_date'], errors='coerce')
employees['termination_date'] = pd.to_datetime(employees['termination_date'], errors='coerce')

In [19]:
# Define reference date for active employees
reference_date = pd.to_datetime('2024-12-31')

In [20]:
# Tenure calculation
employees['tenure_days'] = (
    employees['termination_date'].fillna(reference_date) - employees['hire_date']
).dt.days

employees['tenure_years'] = employees['tenure_days'] / 365

In [21]:
# Merge employees with positions to get department
emp_with_dept = pd.merge(
    employees,
    positions[['id', 'department']],
    left_on='position_id',
    right_on='id',
    how='left',
    suffixes=('', '_pos')
)

# Drop extra column if needed
emp_with_dept.drop(columns=['id_pos'], inplace=True)

In [22]:
tenure_by_dept = (
    emp_with_dept
    .groupby('department')['tenure_years']
    .mean()
    .reset_index()
    .sort_values(by='tenure_years', ascending=False)
)

display(tenure_by_dept.head())

,department,tenure_years
3,Human Resources,11.633425
2,Finance,9.413699
4,Management,8.547945
0,Business Intelligence,8.135642
6,Product Development,4.715215


In [23]:
# Save to CSV
tenure_by_dept.to_csv('../../outputs/summary_tables/tenure_by_department.csv', index=False)

Turnover and Retention Rate

In [ ]:
# Ensure status column is clean
employees['status'] = employees['status'].str.strip().str.lower()

# Create flags
employees['is_active'] = employees['status'] == 'active'
employees['is_terminated'] = employees['termination_date'].notnull()

In [5]:
total_employees = len(employees)
total_terminated = employees['is_terminated'].sum()
total_active = employees['is_active'].sum()

turnover_rate = total_terminated / total_employees
retention_rate = total_active / total_employees

In [6]:
print(f"Total employees: {total_employees}")
print(f"Terminated: {total_terminated} ➜ Turnover rate: {turnover_rate:.2%}")
print(f"Active: {total_active} ➜ Retention rate: {retention_rate:.2%}")

Total employees: 438
Terminated: 277 ➜ Turnover rate: 63.24%
Active: 161 ➜ Retention rate: 36.76%


In [7]:
# Join with positions to get department info
emp_with_dept = pd.merge(
    employees,
    positions[['id', 'department']],
    left_on='position_id',
    right_on='id',
    how='left',
    suffixes=('', '_pos')
)

# Aggregate by department
dept_stats = (
    emp_with_dept.groupby('department')
    .agg(
        total=('id', 'count'),
        terminated=('is_terminated', 'sum'),
        active=('is_active', 'sum')
    )
    .assign(
        turnover_rate=lambda df: df['terminated'] / df['total'],
        retention_rate=lambda df: df['active'] / df['total']
    )
    .reset_index()
    .sort_values(by='turnover_rate', ascending=False)
)

display(dept_stats.head())

,department,total,terminated,active,turnover_rate,retention_rate
6,Product Development,299,216,83,0.722408,0.277592
5,Marketing,33,23,10,0.696970,0.303030
1,Cybersecurity,31,20,11,0.645161,0.354839
2,Finance,10,6,4,0.600000,0.400000
0,Business Intelligence,53,12,41,0.226415,0.773585


In [8]:
dept_stats.to_csv('../outputs/summary_tables/turnover_retention_by_department.csv', index=False)

Salary Analysis

In [9]:
# Merge employment with positions
salary_df = pd.merge(
    employments,
    positions[['id', 'department', 'job_level', 'title']],
    left_on='position_id',
    right_on='id',
    how='left',
    suffixes=('', '_pos')
)

In [10]:
# Salary summary
salary_stats = salary_df['salary'].describe()
print(salary_stats)

count      2575.000000
mean      54745.950753
std       22444.784126
min       30001.000000
25%       40554.015000
50%       47969.970000
75%       59918.340000
max      205058.850000
Name: salary, dtype: float64


In [ ]:
# Average salary by department
salary_by_dept = (
    salary_df
    .groupby('department')['salary']
    .mean()
    .reset_index()
    .sort_values(by='salary', ascending=False)
)

display(salary_by_dept.head())

,department,salary
4,Management,102357.338889
0,Business Intelligence,76985.902511
2,Finance,73710.188081
1,Cybersecurity,56371.509481
3,Human Resources,49815.801167


In [12]:
# Average salary by job level
salary_by_level = (
    salary_df
    .groupby('job_level')['salary']
    .mean()
    .reset_index()
    .sort_values(by='salary', ascending=False)
)

display(salary_by_level.head())

,job_level,salary
0,0,83865.252222
4,4,75596.543978
3,3,66485.746146
2,2,50130.585892
1,1,37899.350578


In [13]:
salary_by_dept.to_csv('../outputs/summary_tables/salary_by_department.csv', index=False)
salary_by_level.to_csv('../outputs/summary_tables/salary_by_job_level.csv', index=False)